In [1]:
import sqlite3
import json

In [41]:
import numpy as np
import pickle

In [6]:
con = sqlite3.connect("/mnt/infonas/data/harshiitb/MTP/MTP/FEVER/FEVEROUS-main/data/feverous_wikiv1.db")

In [7]:
cur = con.cursor()
cur.execute("SELECT id FROM wiki;")

In [8]:
cur.execute("SELECT id FROM wiki;")

In [9]:
titles = cur.fetchall()

In [10]:
titles = np.asarray(titles)

In [11]:
titles

array([[''],
       ['! (Cláudia Pascoal album)'],
       ['! (The Dismemberment Plan album)'],
       ...,
       ['𓈖'],
       ['𝔹'],
       ['𝕊']], dtype='<U186')

In [12]:
data = titles.squeeze()

In [14]:
data[-1]

'𝕊'

In [15]:
#Now we check BLINK here

In [1]:
import blink.main_dense as main_dense
import argparse

models_path = "/mnt/infonas/data/harshiitb/MTP/MTP/BLINK/models/" # the path where you stored the BLINK models

config = {
    "test_entities": None,
    "test_mentions": None,
    "interactive": False,
    "top_k": 10,
    "biencoder_model": models_path+"biencoder_wiki_large.bin",
    "biencoder_config": models_path+"biencoder_wiki_large.json",
    "entity_catalogue": models_path+"entity.jsonl",
    "entity_encoding": models_path+"all_entities_large.t7",
    "crossencoder_model": models_path+"crossencoder_wiki_large.bin",
    "crossencoder_config": models_path+"crossencoder_wiki_large.json",
    "fast": False, # set this to be true if speed is a concern
    "output_path": "logs/" # logging directory
}

args = argparse.Namespace(**config)

models = main_dense.load_models(args, logger=None)

In [8]:
text = '! (Cláudia Pascoal album) Release history Region Portugal Date 27 March 2020 Format  Label Universal Music Portugal'

In [9]:
import blink.ner as NER

In [10]:
ner_model = NER.get_model()

2021-10-12 10:03:22,080 --------------------------------------------------------------------------------
2021-10-12 10:03:22,082 The model key 'ner' now maps to 'https://huggingface.co/flair/ner-english' on the HuggingFace ModelHub
2021-10-12 10:03:22,083  - The most current version of the model is automatically downloaded from there.
2021-10-12 10:03:22,084  - (you can alternatively manually download the original model at https://nlp.informatik.hu-berlin.de/resources/models/ner/en-ner-conll03-v0.4.pt)
2021-10-12 10:03:22,085 --------------------------------------------------------------------------------
2021-10-12 10:03:23,113 loading file /mnt/b100/d0/harshiitb/.flair/models/ner-english/4f4cdab26f24cb98b732b389e6cebc646c36f54cfd6e0b7d3b90b25656e4262f.8baa8ae8795f4df80b28e7f7b61d788ecbb057d1dc85aacb316f1bd02837a4a4


In [11]:
annotated = main_dense._annotate(ner_model,[text])

In [12]:
annotated

[{'label': 'unknown',
  'label_id': -1,
  'context_left': '! (cláudia pascoal album) release history ',
  'context_right': ' date 27 march 2020 format  label universal music portugal',
  'mention': 'region portugal',
  'start_pos': 43,
  'end_pos': 58,
  'sent_idx': 0},
 {'label': 'unknown',
  'label_id': -1,
  'context_left': '! (cláudia pascoal album) release history region portugal date 27 march 2020 format  label ',
  'context_right': '',
  'mention': 'universal music portugal',
  'start_pos': 92,
  'end_pos': 116,
  'sent_idx': 0}]

In [13]:
_, _, _, _, _, predictions, scores, = main_dense.run(args, None, *models, test_data=annotated)

100%|████████████████████████████████████████████| 2/2 [00:00<00:00, 850.86it/s]


Evaluation: 100%|█████████████████████████████████| 2/2 [00:00<00:00,  6.09it/s]


In [15]:
predictions

[['Portugal',
  'Norte Region, Portugal',
  'Centro Region, Portugal',
  'Continental Portugal',
  'Alentejo Region',
  'Autonomous Regions of Portugal',
  'List of regions and sub-regions of Portugal',
  'Greater Portugal',
  'NUTS statistical regions of Portugal',
  'Metropolitan areas in Portugal'],
 ['Universal Music Group',
  'List of Universal Music Group labels',
  'Universal Music Enterprises',
  'Universal Music Publishing Group',
  'Universal Music Distribution',
  'Calderstone Productions',
  'Universal Music TV',
  'Universal Music Australia',
  'Universal Music Latin Entertainment',
  'Universal Music Polska']]

In [17]:
#Now try ELQ

In [1]:
import elq.main_dense as elq_main_dense
import sys
import argparse

In [7]:
sys.path.append("/mnt/infonas/data/harshiitb/MTP/MTP/BLINK")

In [2]:
sys.path

['/mnt/b100/d0/harshiitb/FEVEREXTR/FEVEROUS',
 '/mnt/infonas/data/harshiitb/envs/env1/lib/python39.zip',
 '/mnt/infonas/data/harshiitb/envs/env1/lib/python3.9',
 '/mnt/infonas/data/harshiitb/envs/env1/lib/python3.9/lib-dynload',
 '',
 '/mnt/infonas/data/harshiitb/envs/env1/lib/python3.9/site-packages',
 '/mnt/infonas/data/harshiitb/MTP/MTP/BLINK/src/blink',
 '/mnt/infonas/data/harshiitb/envs/env1/lib/python3.9/site-packages/IPython/extensions',
 '/mnt/b100/d0/harshiitb/.ipython']

In [12]:
models_path = "/mnt/infonas/data/harshiitb/MTP/MTP/BLINK/models/"

In [10]:
elq_config = {
    "interactive": False,
    "biencoder_model": models_path+"elq_wiki_large.bin",
    "biencoder_config": models_path+"elq_large_params.txt",
    "cand_token_ids_path": models_path+"entity_token_ids_128.t7",
    "entity_catalogue": models_path+"entity.jsonl",
    "entity_encoding": models_path+"all_entities_large.t7",
    "output_path": "logs/", # logging directory
    "faiss_index": "hnsw",
    "index_path": models_path+"faiss_hnsw_index.pkl",
    "num_cand_mentions": 10,
    "num_cand_entities": 10,
    "threshold_type": "joint",
    "threshold": -4.5,
}

elq_args = argparse.Namespace(**elq_config)

models = elq_main_dense.load_models(elq_args, logger=None)

FileNotFoundError: [Errno 2] No such file or directory: 'models/id2title.json'

In [12]:
#Now Let's check high IDF

In [1]:
import pandas as pd

In [2]:
dataframe = pd.read_csv("/mnt/infonas/data/harshiitb/MTP/MTP/data/wiki_tfidf_stems.csv")

In [42]:
import snowballstemmer
stemmer = snowballstemmer.stemmer('english')

In [3]:
dataframe[:5]

,stem,frequency,total,idf,most_freq_term
0,the,5457571,5989879,0.093067,the
1,in,5234520,5989879,0.134796,in
2,a,5182913,5989879,0.144704,a
3,of,4997632,5989879,0.181107,of
4,is,4655043,5989879,0.252120,is


In [67]:
s = stemmer.stemWord("nominated")

In [68]:
print(s)

nomin


In [28]:
df = dataframe[dataframe['idf']<=7]
df = df[df['idf']>=3]

In [29]:
df = df[["stem","most_freq_term"]]

In [30]:
df

,stem,most_freq_term
522,21,21
523,televis,television
524,exampl,example
525,church,church
526,individu,individual
...,...,...
10940,schoolboy,schoolboy
10941,massif,massif
10942,mia,mia
10943,brahmin,brahmin


In [37]:
rare_words = df.set_index('stem').T.to_dict('list')

In [39]:
rare_words

{'21': ['21'],
 'televis': ['television'],
 'exampl': ['example'],
 'church': ['church'],
 'individu': ['individual'],
 'inform': ['information'],
 'rang': ['range'],
 'u.s.': ['u.s.'],
 'student': ['students'],
 'select': ['selected'],
 'taken': ['taken'],
 'wide': ['wide'],
 '22': ['22'],
 'notabl': ['notable'],
 '19': ['19'],
 'claim': ['claimed'],
 'daughter': ['daughter'],
 'limit': ['limited'],
 '2002': ['2002'],
 'version': ['version'],
 'georg': ['george'],
 'seven': ['seven'],
 'track': ['track'],
 'write': ['writing'],
 'societi': ['society'],
 'carri': ['carried'],
 'german': ['german'],
 'find': ['find'],
 'franc': ['france'],
 '2001': ['2001'],
 'littl': ['little'],
 'post': ['post'],
 'despit': ['despite'],
 'king': ['king'],
 'separ': ['separate'],
 '100': ['100'],
 'signific': ['significant'],
 'connect': ['connected'],
 'competit': ['competition'],
 'full': ['full'],
 '23': ['23'],
 'process': ['process'],
 'modern': ['modern'],
 'doe': ['does'],
 'avail': ['available'

In [40]:
#Now load the arae_word.pkl

In [44]:
with open("/mnt/infonas/data/harshiitb/MTP/MTP/data/rare_words.pkl", "rb") as file:
    rw = pickle.load(file)

In [46]:
len(rw.keys())

20367